In [3]:
import os
import subprocess
import logging
from datetime import datetime
from mutagen.flac import FLAC
import time

In [ ]:
# Configuration
MUSIC_DIR = "C:\\Users\\Max\\Desktop\\music"  # ← Change this to your actual folder
LOG_FILE = "lyrics_fetch_log.txt"

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(LOG_FILE, encoding='utf-8'),
        logging.StreamHandler()  # also print to console
    ]
)

def main():
    logging.info("===== Starting synced lyrics fetcher (with album in search) =====")
    logging.info(f"Music directory: {MUSIC_DIR}")

    if not os.path.isdir(MUSIC_DIR):
        logging.error(f"Directory not found: {MUSIC_DIR}")
        return

    flac_files = [f for f in os.listdir(MUSIC_DIR) if f.lower().endswith(".flac")]
    total_files = len(flac_files)
    logging.info(f"Found {total_files} FLAC files")

    processed = 0
    success = 0
    skipped = 0

    for i, filename in enumerate(flac_files, 1):
        file_path = os.path.join(MUSIC_DIR, filename)
        try:
            audio = FLAC(file_path)
            artist = audio.get("artist", [""])[0].strip()
            title = audio.get("title", [""])[0].strip()
            album = audio.get("album", [""])[0].strip()

            if not (artist and title):
                logging.warning(f"[{i}/{total_files}] Skipping {filename} - missing artist or title tag")
                skipped += 1
                continue

            processed += 1

            # Build search query: include album if available for better accuracy
            if album:
                search_query = f'"{title} {artist} {album}"'
                logging.info(f"[{i}/{total_files}] Processing: {filename} → Searching '{search_query}' (with album)")
            else:
                search_query = f'"{title} {artist}"'
                logging.info(f"[{i}/{total_files}] Processing: {filename} → Searching '{search_query}' (no album tag)")

            lrc_path = os.path.join(MUSIC_DIR, os.path.splitext(filename)[0] + ".lrc")

            # Run syncedlyrics command
            result = subprocess.run(
                ["syncedlyrics", search_query, "-o", lrc_path],
                capture_output=True,
                text=True,
                timeout=60
            )

            if result.returncode == 0 and os.path.exists(lrc_path) and os.path.getsize(lrc_path) > 10:
                logging.info(f"Success: Lyrics saved to {os.path.basename(lrc_path)}")
                success += 1
            else:
                logging.warning(f"No synced lyrics found for query '{search_query}'")
                if result.stderr:
                    logging.debug(f"syncedlyrics error: {result.stderr.strip()}")

            # Polite delay
            time.sleep(0.5)

        except Exception as e:
            logging.error(f"Error processing {filename}: {str(e)}")

    # Final summary
    if processed > 0:
        success_rate = (success / processed) * 100
    else:
        success_rate = 0.0

    logging.info("===== Finished processing all files =====")
    logging.info(f"Summary:")
    logging.info(f"  Total FLAC files: {total_files}")
    logging.info(f"  Processed: {processed} (skipped {skipped} due to missing tags)")
    logging.info(f"  Lyrics found: {success}/{processed} ({success_rate:.1f}% success rate)")
    logging.info(f"Log saved to: {os.path.abspath(LOG_FILE)}")    



In [9]:
main()

2025-12-25 23:56:12,195 - INFO - ===== Starting synced lyrics fetcher (with album in search) =====
2025-12-25 23:56:12,197 - INFO - Music directory: F:\Media\Music
2025-12-25 23:56:12,201 - INFO - Found 89 FLAC files
2025-12-25 23:56:12,215 - INFO - [1/89] Processing: 01 Ed Sheeran - Sapphire.flac → Searching '"Sapphire Ed Sheeran Sapphire"' (with album)
Exception in thread Thread-7 (_readerthread):
Traceback (most recent call last):
  File "c:\Users\Max\AppData\Local\Programs\Python\Python314\Lib\threading.py", line 1082, in _bootstrap_inner
    self._context.run(self.run)
    ~~~~~~~~~~~~~~~~~^^^^^^^^^^
  File "c:\Users\Max\AppData\Local\Programs\Python\Python314\Lib\threading.py", line 1024, in run
    self._target(*self._args, **self._kwargs)
    ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Max\AppData\Local\Programs\Python\Python314\Lib\subprocess.py", line 1613, in _readerthread
    buffer.append(fh.read())
                  ~~~~~~~^^
  File "c:\Users\Max\AppData\Lo